In [122]:
import pandas as pd
from time import sleep
import json

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [240]:
supplements = pd.read_csv("Supplements.csv")
supplements.head(5)

,Name,Url
0,Abscess Root,https://www.webmd.com/vitamins/ai/ingredientmo...
1,Abuta,https://www.webmd.com/vitamins/ai/ingredientmo...
2,Acacia Rigidula,https://www.webmd.com/vitamins/ai/ingredientmo...
3,Acai,https://www.webmd.com/vitamins/ai/ingredientmo...
4,Acerola,https://www.webmd.com/vitamins/ai/ingredientmo...


In [241]:
supplements = supplements[supplements["Name"].str.startswith('Cannabis')]
supplements


,Name,Url
261,Cannabis,https://www.webmd.com/vitamins/ai/ingredientmo...


In [242]:
def get_uses(elementPath):
    uses = {}

    try:
        element = wait_time.until(EC.presence_of_element_located((By.XPATH, elementPath)))
        div_elements = element.find_elements(By.CLASS_NAME, "vitamins-monograph-content uses-content")
        

        h3_tags = element.find_elements(By.TAG_NAME, "h3")
        
        ul_counter = 1
        for h3 in h3_tags:
            uses_per_tag = []

            ul_path = f"/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]/div[{ul_counter}]/ul"
            ul_elements = element.find_elements(By.XPATH, ul_path)

            if ul_elements:
                li_elements = ul_elements[0].find_elements(By.TAG_NAME, "li")
                
                for item in li_elements:
                    li_text = item.text.strip() 
                    uses_per_tag.append([li_text])

                uses[h3.text] = uses_per_tag
            ul_counter += 1

    except Exception as e:
        uses["Error"] = e
    finally:
        return uses

In [ ]:
def get_overview_content():
    overview = {}
    try:
        
        overview_container = wait_time.until(EC.presence_of_element_located((By.XPATH, "//*[@id=\"overview-container\"]")))
        
        try:
            about_container = overview_container.find_elements(By.XPATH, "./div[1]/div[1]")
            for about in about_container:
                overview["About"] = about.text.replace("\n\n"," ").strip()
        except Exception:
            overview["About"] = "No information found" 

        # HOW TO USE
        try:
            howToUse_container = overview_container.find_element(By.XPATH, "//*[@id=\"overview-container\"]/div[1]/div[2]")
            howToUse_child = howToUse_container.find_elements(By.XPATH, ".//*")
            for howTo in howToUse_child:
                print(howTo.tag_name)
                if howTo.tag_name.strip() == "div":
                    overview["How To Use"] = howTo.text.replace("\n\n"," ").strip()
        except Exception:
            overview["About"] = "No details on how to use" 

    except Exception:
        overview.clear
        overview["Error"] = ""
    finally:
        return overview

In [244]:
def uses_content():
    uses = {}

In [245]:
supplement_info = {}

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait_time = WebDriverWait(driver,10)

for _, info in supplements.head(1).iterrows():

    supplementName = info.Name
    supplementURL = info.Url
    
    driver.get(supplementURL)       

    overview = get_overview_content()
    print(overview)
    #overview_content = parent_monograph.find_elements(By.CLASS_NAME, "vitamins-monograph-content overview-content")
    #text = overview_content.text.strip()
    
    
    #overview = get_overview_content()
    #overview = get_element_text(supplementName, "Overview", "/html/body/div[1]/div/main/div/div[2]/div/div/div[1]/div[1]/div")
    #uses = get_uses("/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]")
    #sideEffects = get_element_text(supplementName, "Side Effects", "/html/body/div[1]/div/main/div/div[2]/div/div/div[3]/div[1]")
    #precautions = get_element_text(supplementName, "Precautions", "/html/body/div[1]/div/main/div/div[2]/div/div/div[4]/div[1]/div")
    #interactons = [] /html/body/div[1]/div/main/div/div[2]/div/div/div[4]/div[2]/div
    #dosing = get_element_text(supplementName, "Dosing", "/html/body/div[1]/div/main/div/div[2]/div/div/div[6]/div[1]/div")

    # supplement_info[supplementName] = {
    #     "Overview": overview,
    #     "Uses": [],
    #     "SideEffects": [],
    #     "Precautions": [],
    #     "Interactions": [],
    #     "Dosing": [] 
    # }
    sleep(5)
driver.quit()

{'About': "Cannabis (Cannabis sativa) is an herbal drug. It contains chemicals called cannabinoids, including delta-9-tetrahydrocannabinol (THC) and cannabidiol (CBD). The cannabinoids in cannabis work by binding to specific sites in the brain and on the nerves. There are over 100 cannabinoids in cannabis, but THC and CBD are the most well-studied. Cannabinoids are found in the highest levels in the leaves and flowers of the plant. Cannabis is commonly used as a recreational drug. People also commonly use cannabis for multiple sclerosis (MS) and nerve pain. It is also used for nausea, vomiting, migraine, and many other conditions, but there is no good scientific evidence to support these uses. There is also no good evidence to support using cannabis for COVID-19. Don't confuse cannabis with hemp. Hemp contains very low levels of THC, less than 0.3% according to legal standards. Both hemp and cannabis also contain cannabinoids such as CBD, cannabidivarin (CBDV), cannabigerol (CBG), and 

In [136]:
print(json.dumps(supplement_info, indent=4))

{
    "Abscess Root": {
        "Overview": "Abscess root (Polemonium reptans) is an herb. The root is ground and used as a medicinal tea, but it's not clear if it has any benefits.\n\nAbscess root is used for fever, cough, swelling, and other conditions. But there is no good scientific evidence to support any uses.\n\nAbscess root is also called \"false Jacob's ladder,\" because it has some of the same effects as another plant called Jacob's ladder. These two plants are used in similar ways, but are not the same.",
        "Uses": [],
        "SideEffects": [],
        "Precautions": [],
        "Interactions": [],
        "Dosing": []
    }
}


TEST HERE

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.webmd.com/vitamins/ai/ingredientmono-266/abscess-root")
wait_time = WebDriverWait(driver,5)
#div = "/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]"
/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]/div[3]
div = "/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/p"
elem = wait_time.until(EC.presence_of_element_located((By.XPATH, div)))
elem = wait_time.until(EC.)
print(elem.text)
driver.quit()